In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

In [3]:
%cd "Documents/GitHub/pmx_gasoline_theft_prediction/"

/Users/jorgeroa/Documents/GitHub/pmx_gasoline_theft_prediction


In [89]:
df_homics_15_19 = pd.read_parquet("data/df_homics_15_19.parquet")
df_PMX_15_19_thefts_pp = pd.read_parquet('data/df_PMX_15_19_thefts_pp_final.parquet')
df_PMX_gas_place_price = pd.read_parquet('data/df_places_prices_final.parquet')
df_social_mean_2015_2019 = pd.read_excel('data/df_social_15_19_final_JR.xlsx')
df_SEDENA_15_19 = pd.read_parquet('data/df_SEDENA_15_19.parquet')
df_cvegeo = pd.read_excel("data-raw/codigo municipio inegi.xlsx")
df_cvegeo = df_cvegeo.rename(columns = {"codigo inegi":"CVEGEO"})

df_cvegeo['CVEGEO'] = df_cvegeo['CVEGEO'].astype(str).str.zfill(7).str.replace('.',"")
df_cvegeo['CVEGEO'] = df_cvegeo['CVEGEO'].str[:-1]
df_cvegeo_f = df_cvegeo[["CVEGEO", "NOM_ENT", "NOM_MUN"]]

df_social_mean_2015_2019['CVEGEO']=df_social_mean_2015_2019['CVEGEO'].astype(object)
df_social_mean_2015_2019['CVEGEO'] = df_social_mean_2015_2019['CVEGEO'].astype(str).str.zfill(5)

df_PMX_15_19_thefts_pp

/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_21328/4097619903.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_cvegeo['CVEGEO'] = df_cvegeo['CVEGEO'].astype(str).str.zfill(7).str.replace('.',"")


,CVEGEO,type_inegi,soil_inegi,pp_inegi,cap_5ypp,inv_5ypp,long_5ypp,pp_5ypp,pp_instal,pp_integ_sistran_pp,...,long_SISTRANGAS_pipeline,pp_SISTRANGAS_pipeline,pp_st_term,type_st_term,cap_st_term,cap_st_term_1,pipeline,sum_intersects,Num_thefts,_merge
0,09012,UN CONDUCTO,SUBTERRANEO,2.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,1.0,Poliducto,23000.0,22000.0,1.0,4.0,37.0,both
1,09013,UN CONDUCTO,SUBTERRANEO,3.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,1.0,Poliducto,23000.0,22000.0,1.0,5.0,NaN,left_only
2,09008,UN CONDUCTO,SUBTERRANEO,1.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,1.0,1.0,NaN,left_only
3,09002,UN CONDUCTO,SUBTERRANEO,3.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,5.0,Turbosino-ducto,428000.0,412000.0,1.0,13.0,5.0,both
4,09014,None,None,0.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,0.0,0.0,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,23003,None,None,0.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,0.0,0.0,NaN,left_only
2453,15108,UN CONDUCTO,SUBTERRANEO,2.0,None,None,NaN,0.0,0.0,1.0,...,NaN,1.0,0.0,None,NaN,NaN,1.0,4.0,1.0,both
2454,15059,UN CONDUCTO,SUBTERRANEO,2.0,None,None,NaN,0.0,0.0,1.0,...,NaN,1.0,0.0,None,NaN,NaN,1.0,4.0,22.0,both
2455,15109,UN CONDUCTO,SUBTERRANEO,6.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,1.0,6.0,12.0,both


In [90]:
df_cvegeo_f

,CVEGEO,NOM_ENT,NOM_MUN
0,01001,Aguascalientes,Aguascalientes
1,01002,Aguascalientes,Asientos
2,01003,Aguascalientes,Calvillo
3,01004,Aguascalientes,Cosío
4,01005,Aguascalientes,Jesús María
...,...,...,...
2451,32054,Zacatecas,Villa Hidalgo
2452,32055,Zacatecas,Villanueva
2453,32056,Zacatecas,Zacatecas
2454,32057,Zacatecas,Trancoso


In [66]:
df_homics_15_19.dtypes

CVEGEO          object
homic_15_19    float64
dtype: object

In [54]:
df_PMX_15_19_thefts_pp.dtypes

CVEGEO                        object
type_inegi                    object
soil_inegi                    object
pp_inegi                     float64
cap_5ypp                      object
inv_5ypp                      object
long_5ypp                    float64
pp_5ypp                      float64
pp_instal                    float64
pp_integ_sistran_pp          float64
long_integ_sistran_pp        float64
type_integ_sistran_pp         object
pp_mix_pp                    float64
long_mix_pp                  float64
type_tube_mix_pp              object
type_gas_mix_pp               object
pp_n_int_sist                float64
long_n_int_sist              float64
type_n_int_sist               object
cap_n_int_sist               float64
inv_n_int_sist               float64
pp_oil                       float64
long_oil                     float64
type_oil                      object
cap_oil                      float64
cap_oil_2                    float64
cap_pp_tran_sys               object
i

In [55]:
df_PMX_15_19_thefts_pp

,CVEGEO,type_inegi,soil_inegi,pp_inegi,cap_5ypp,inv_5ypp,long_5ypp,pp_5ypp,pp_instal,pp_integ_sistran_pp,...,long_SISTRANGAS_pipeline,pp_SISTRANGAS_pipeline,pp_st_term,type_st_term,cap_st_term,cap_st_term_1,pipeline,sum_intersects,Num_thefts,_merge
0,09012,UN CONDUCTO,SUBTERRANEO,2.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,1.0,Poliducto,23000.0,22000.0,1.0,4.0,37.0,both
1,09013,UN CONDUCTO,SUBTERRANEO,3.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,1.0,Poliducto,23000.0,22000.0,1.0,5.0,NaN,left_only
2,09008,UN CONDUCTO,SUBTERRANEO,1.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,1.0,1.0,NaN,left_only
3,09002,UN CONDUCTO,SUBTERRANEO,3.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,5.0,Turbosino-ducto,428000.0,412000.0,1.0,13.0,5.0,both
4,09014,None,None,0.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,0.0,0.0,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,23003,None,None,0.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,0.0,0.0,NaN,left_only
2453,15108,UN CONDUCTO,SUBTERRANEO,2.0,None,None,NaN,0.0,0.0,1.0,...,NaN,1.0,0.0,None,NaN,NaN,1.0,4.0,1.0,both
2454,15059,UN CONDUCTO,SUBTERRANEO,2.0,None,None,NaN,0.0,0.0,1.0,...,NaN,1.0,0.0,None,NaN,NaN,1.0,4.0,22.0,both
2455,15109,UN CONDUCTO,SUBTERRANEO,6.0,None,None,NaN,0.0,0.0,0.0,...,NaN,0.0,0.0,None,NaN,NaN,1.0,6.0,12.0,both


In [91]:


dataframes = [df_cvegeo_f,
              df_PMX_15_19_thefts_pp,
                    df_PMX_gas_place_price,
                    df_homics_15_19,
                    df_SEDENA_15_19,
                    df_social_mean_2015_2019]

df_PMX_thefts_15_19 = reduce(lambda  left,right: pd.merge(left,right,on=['CVEGEO'],
                                            how = 'outer'), dataframes)#.fillna(0)

In [92]:
df_PMX_thefts_15_19_pp = df_PMX_thefts_15_19[(df_PMX_thefts_15_19['pipeline']==1) & (df_PMX_thefts_15_19['CVEGEO'].str.len()>=5)]

In [62]:
df_homics_15_19

,CVEGEO,homic_15_19
0,01001,49.0
1,01002,2.0
2,01003,3.0
3,01004,1.0
4,01005,6.0
...,...,...
2486,07124,2.0
2487,05999,1.0
2488,21999,0.0
2489,27999,0.0


In [93]:
df_PMX_thefts_15_19_pp["homic_rate"] = df_PMX_thefts_15_19_pp["homic_15_19"]/df_PMX_thefts_15_19_pp["POB_TOT"]*100000

/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_21328/1132246183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PMX_thefts_15_19_pp["homic_rate"] = df_PMX_thefts_15_19_pp["homic_15_19"]/df_PMX_thefts_15_19_pp["POB_TOT"]*100000


In [104]:

df_PMX_thefts_15_19_pp['confront']=df_PMX_thefts_15_19_pp['confront'].fillna(0)

df_PMX_thefts_15_19_pp_f = df_PMX_thefts_15_19_pp.drop(['_merge_x', '_merge_y', "pipeline"], axis=1)


df_PMX_thefts_15_19_pp_f['pipeline'] = np.select([(df_PMX_thefts_15_19_pp_f['Num_thefts'] >= 0)], [1])



df_PMX_thefts_15_19_pp_f['pipeline'].sum()

/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_21328/1784232906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PMX_thefts_15_19_pp['confront']=df_PMX_thefts_15_19_pp['confront'].fillna(0)


426

In [105]:
df_PMX_thefts_15_19_pp_f.to_excel('data/df_PMX_thefts_15_19_pp_final.xlsx', index = False)
df_PMX_thefts_15_19_pp_f.to_parquet('data/df_PMX_thefts_15_19_pp_final.parquet')